In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/Train (2).csv')
test_given = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/Test (2).csv')
sample_submission = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/SampleSubmission (2).csv')

In [3]:
# Encoding categorical variables
train['form_field47'].replace('charge',1, inplace=True)
train['form_field47'].replace('lending',0, inplace=True)
train['default_status'].replace('yes',1, inplace=True)
train['default_status'].replace('no',0, inplace=True)

test_given['form_field47'].replace('charge',1, inplace=True)
test_given['form_field47'].replace('lending',0, inplace=True)


In [4]:
new_features = train.select_dtypes(exclude = object).columns.drop(['default_status','form_field23','form_field45','form_field11', 'form_field4','form_field17','form_field18','form_field41','form_field40','form_field30','form_field31','form_field35','form_field39'])
test_new_features = test_given.select_dtypes(exclude = object).columns.drop(['form_field23','form_field45','form_field11','form_field4','form_field17','form_field18','form_field30','form_field31','form_field35','form_field39','form_field41','form_field40'])

In [5]:
for i in new_features:
    train[i].fillna(-999, inplace=True)
    test_given[i].fillna(-999, inplace=True)

In [6]:
test_given_new = test_given[test_new_features]
test_given_new.head()

,form_field1,form_field2,form_field3,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field12,...,form_field37,form_field38,form_field42,form_field43,form_field44,form_field46,form_field47,form_field48,form_field49,form_field50
0,3236.0,0.34875,10.2006,0.0,418564.0,418564.0,418564.0,540710.0,0.0,5446.0,...,2.0,14.0,0.825000,1.01,0.800000,0.0,1,-999.000000,0.000000,0.011221
1,3284.0,1.27360,2.9606,0.0,0.0,9858816.0,49014.0,1510098.0,18308285.0,1025793.0,...,18.0,14.0,0.507694,4.04,0.623248,0.0,0,-999.000000,0.504974,0.043525
2,-999.0,0.27505,0.0600,0.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,...,-999.0,0.0,-999.000000,0.00,-999.000000,-999.0,1,-999.000000,0.000000,-999.000000
3,3232.0,0.28505,2.8032,0.0,0.0,473802.0,473802.0,1724437.0,493641.0,1176725.0,...,0.0,26.0,0.916663,2.02,0.464224,-999.0,1,90.163742,0.788809,0.104029
4,3466.0,2.09545,0.8318,0.0,19839.0,1150662.0,1150662.0,7860523.0,5752921.0,613064.0,...,28.0,20.0,0.234047,23.23,0.726688,0.0,0,1303.587148,1.637733,0.163124


In [25]:
#train[new_features].isna().sum()

In [26]:
X = train[new_features]
Y = train.iloc[:,-1]

In [9]:
from sklearn.model_selection import StratifiedKFold

n_splits = 5
params = {'n_estimators':4000, 'learning_rate':0.01, 'objective':'CrossEntropy','eval_metric':'AUC',
          'random_seed': 1234,'early_stopping_rounds':200,'use_best_model':True}
fold=StratifiedKFold(n_splits)


In [10]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
error=[]
test_pred=[]
score = 0 
i=1
for train_index, test_index in fold.split(X,Y):
    
    #The usual cross validation
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    model  = CatBoostClassifier(**params)
    model.fit(X_train,Y_train,eval_set=[(X_train,Y_train),(X_test, Y_test)], early_stopping_rounds=300,verbose=100,)#erly100
    pred = model.predict(X_test)
    print("error: ", accuracy_score(Y_test,pred))
    error.append(accuracy_score(Y_test,pred))
    
    #Simultaneosly Predicting on test set for submission
    pred_sub = model.predict(test_given_new)
    test_pred.append(pred_sub)
    preds_proba = model.predict_proba(test_given_new)[:,1]
    
np.mean(error)
print(error)

0:	test: 0.7886084	test1: 0.7890079	best: 0.7890079 (0)	total: 163ms	remaining: 10m 51s
100:	test: 0.8282710	test1: 0.8242924	best: 0.8242924 (100)	total: 6.98s	remaining: 4m 29s
200:	test: 0.8336065	test1: 0.8282713	best: 0.8282713 (200)	total: 12.4s	remaining: 3m 53s
300:	test: 0.8370959	test1: 0.8307159	best: 0.8307159 (300)	total: 17.1s	remaining: 3m 30s
400:	test: 0.8396919	test1: 0.8322014	best: 0.8322014 (400)	total: 21.8s	remaining: 3m 16s
500:	test: 0.8418775	test1: 0.8333543	best: 0.8333543 (500)	total: 26.8s	remaining: 3m 7s
600:	test: 0.8439060	test1: 0.8342736	best: 0.8342793 (599)	total: 31.7s	remaining: 2m 59s
700:	test: 0.8457559	test1: 0.8349921	best: 0.8349921 (700)	total: 36.5s	remaining: 2m 51s
800:	test: 0.8476200	test1: 0.8356795	best: 0.8356795 (800)	total: 42.3s	remaining: 2m 49s
900:	test: 0.8493320	test1: 0.8362456	best: 0.8362456 (900)	total: 47.1s	remaining: 2m 42s
1000:	test: 0.8509555	test1: 0.8367188	best: 0.8367188 (1000)	total: 51.9s	remaining: 2m 35s
1

2900:	test: 0.8817867	test1: 0.8425620	best: 0.8425648 (2623)	total: 2m 32s	remaining: 57.8s
3000:	test: 0.8831457	test1: 0.8425601	best: 0.8426034 (2940)	total: 2m 37s	remaining: 52.6s
3100:	test: 0.8844173	test1: 0.8425389	best: 0.8426034 (2940)	total: 2m 43s	remaining: 47.3s
3200:	test: 0.8857681	test1: 0.8425318	best: 0.8426034 (2940)	total: 2m 48s	remaining: 42s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.8426034283
bestIteration = 2940

Shrink model to first 2941 iterations.
error:  0.8103571428571429
0:	test: 0.7926422	test1: 0.7738344	best: 0.7738344 (0)	total: 97.2ms	remaining: 6m 28s
100:	test: 0.8299793	test1: 0.8145529	best: 0.8145529 (100)	total: 6.15s	remaining: 3m 57s
200:	test: 0.8355733	test1: 0.8200243	best: 0.8200243 (200)	total: 11.1s	remaining: 3m 29s
300:	test: 0.8392222	test1: 0.8227974	best: 0.8227974 (300)	total: 16s	remaining: 3m 16s
400:	test: 0.8418841	test1: 0.8244304	best: 0.8244304 (400)	total: 20.9s	remaining: 3m 7s
500:	test: 0.

In [11]:
preds_proba

array([0.38451635, 0.3283575 , 0.33719225, ..., 0.28567967, 0.49854218,
       0.21197174])

In [13]:
#file for submission
id = test_given['Applicant_ID']
#id

sub = pd.DataFrame(id, columns=['Applicant_ID'])
sub['default_status'] = preds_proba
sub

sub.to_csv('DSN_Hack(cat-999stra5).csv', index=False)

In [9]:
#Splitting into test and train set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state = 0)

In [10]:
from sklearn.metrics import roc_auc_score
def metric(Y,pred_test):
    return roc_auc_score(Y,pred_test, labels=[0,1])

In [12]:
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import StratifiedKFold

params = {'n_estimators':4000, 'learning_rate':0.01, 'objective':'CrossEntropy','eval_metric':'AUC',
          'random_seed': 1234,'early_stopping_rounds':200,'use_best_model':True}

n_fold = 15
kf = StratifiedKFold(n_fold)

list_scores = []
test_oofs = []
score = 0 

for i, (tr_idx, vr_idx) in enumerate(kf.split(X,Y)):
    X_train, Y_train = X.loc[tr_idx, new_features], Y.loc[tr_idx]
    X_val, Y_val = X.loc[vr_idx,new_features], Y.loc[vr_idx]
    
    model= CatBoostClassifier(**params)
    model.fit(X_train,Y_train,eval_set=[(X_val,Y_val),(X_test, Y_test)],verbose=100,)#erly100
    
    pred = model.predict_proba(X_val)[:,1]
    sc = metric(Y_val, pred)
    list_scores.append(sc)
    score += sc/n_fold
    
    pred_test = model.predict_proba(test_given_new[test_new_features])[:,1]
    test_oofs.append(pred)

0:	test: 0.7883275	test1: 0.7882349	best: 0.7882349 (0)	total: 212ms	remaining: 14m 9s
100:	test: 0.8153027	test1: 0.8226723	best: 0.8226723 (100)	total: 7.19s	remaining: 4m 37s
200:	test: 0.8192579	test1: 0.8285886	best: 0.8285886 (200)	total: 12.8s	remaining: 4m 2s
300:	test: 0.8219025	test1: 0.8323516	best: 0.8323516 (300)	total: 19.2s	remaining: 3m 55s


KeyboardInterrupt: 

In [25]:
#test_oofs
oof_prediction = pd.DataFrame(test_oofs).T


sample_submission['default_status'] = pred_test
sample_submission.to_csv('DSN_Hack_New(cat-999stra15-).csv')

In [26]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
error=[]
test_pred=[]
score = 0 
i=1
for train_index, test_index in fold.split(X,Y):
    
    #The usual cross validation
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    model  = CatBoostClassifier(**params)
    model.fit(X_train,Y_train,eval_set=[(X_train,Y_train),(X_test, Y_test)],verbose=100)
    pred = model.predict(X_test)
    print("error: ", accuracy_score(Y_test,pred))
    error.append(accuracy_score(Y_test,pred))
    
    #Simultaneosly Predicting on test set for submission
    pred_sub = model.predict(test_given_new)
    test_pred.append(pred_sub)
    preds_proba = model.predict_proba(test_given_new)[:,1]
    
np.mean(error)
print(error)

0:	test: 0.7937454	test1: 0.7883275	best: 0.7883275 (0)	total: 147ms	remaining: 9m 46s
100:	test: 0.8286844	test1: 0.8153027	best: 0.8153027 (100)	total: 7.37s	remaining: 4m 44s
200:	test: 0.8338272	test1: 0.8192579	best: 0.8192579 (200)	total: 12.9s	remaining: 4m 3s
300:	test: 0.8371343	test1: 0.8219025	best: 0.8219025 (300)	total: 19.1s	remaining: 3m 54s
400:	test: 0.8396341	test1: 0.8235134	best: 0.8235153 (399)	total: 24.4s	remaining: 3m 38s
500:	test: 0.8417288	test1: 0.8248916	best: 0.8249036 (499)	total: 29.9s	remaining: 3m 28s
600:	test: 0.8436228	test1: 0.8258965	best: 0.8258965 (600)	total: 35.7s	remaining: 3m 21s
700:	test: 0.8453737	test1: 0.8268552	best: 0.8268583 (699)	total: 41.6s	remaining: 3m 15s
800:	test: 0.8470715	test1: 0.8276415	best: 0.8276415 (800)	total: 47.8s	remaining: 3m 10s
900:	test: 0.8486037	test1: 0.8281935	best: 0.8281986 (898)	total: 54.1s	remaining: 3m 5s
1000:	test: 0.8501647	test1: 0.8287805	best: 0.8287844 (996)	total: 1m	remaining: 3m
1100:	test:

1600:	test: 0.8594369	test1: 0.8394513	best: 0.8394551 (1584)	total: 2m	remaining: 2m 59s
1700:	test: 0.8610105	test1: 0.8393478	best: 0.8394567 (1627)	total: 2m 6s	remaining: 2m 51s
1800:	test: 0.8625305	test1: 0.8392489	best: 0.8394567 (1627)	total: 2m 13s	remaining: 2m 43s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8394566922
bestIteration = 1627

Shrink model to first 1628 iterations.
error:  0.807980717728977
0:	test: 0.7884738	test1: 0.7950938	best: 0.7950938 (0)	total: 129ms	remaining: 8m 37s
100:	test: 0.8278041	test1: 0.8334056	best: 0.8334056 (100)	total: 7.67s	remaining: 4m 56s
200:	test: 0.8326637	test1: 0.8382845	best: 0.8382845 (200)	total: 14.3s	remaining: 4m 30s
300:	test: 0.8360125	test1: 0.8409287	best: 0.8409287 (299)	total: 21s	remaining: 4m 17s
400:	test: 0.8384736	test1: 0.8426875	best: 0.8426883 (399)	total: 27.6s	remaining: 4m 7s
500:	test: 0.8406734	test1: 0.8439343	best: 0.8439343 (500)	total: 34.3s	remaining: 3m 59s
600:	test: 0.8425

1000:	test: 0.8493929	test1: 0.8404489	best: 0.8404489 (1000)	total: 1m 8s	remaining: 3m 25s
1100:	test: 0.8509860	test1: 0.8410489	best: 0.8410702 (1098)	total: 1m 15s	remaining: 3m 18s
1200:	test: 0.8526955	test1: 0.8414642	best: 0.8414677 (1198)	total: 1m 22s	remaining: 3m 11s
1300:	test: 0.8544621	test1: 0.8417987	best: 0.8417987 (1300)	total: 1m 28s	remaining: 3m 4s
1400:	test: 0.8561750	test1: 0.8420727	best: 0.8420848 (1399)	total: 1m 35s	remaining: 2m 57s
1500:	test: 0.8578930	test1: 0.8424166	best: 0.8424209 (1498)	total: 1m 42s	remaining: 2m 50s
1600:	test: 0.8595585	test1: 0.8425206	best: 0.8425210 (1596)	total: 1m 49s	remaining: 2m 43s
1700:	test: 0.8611561	test1: 0.8424403	best: 0.8425556 (1607)	total: 1m 56s	remaining: 2m 36s
1800:	test: 0.8626754	test1: 0.8425420	best: 0.8425618 (1794)	total: 2m 2s	remaining: 2m 29s
1900:	test: 0.8641566	test1: 0.8425396	best: 0.8425618 (1794)	total: 2m 9s	remaining: 2m 23s
2000:	test: 0.8656144	test1: 0.8425478	best: 0.8425761 (1920)	to

500:	test: 0.8424880	test1: 0.8146723	best: 0.8146723 (500)	total: 34.7s	remaining: 4m 2s
600:	test: 0.8443351	test1: 0.8155382	best: 0.8155382 (600)	total: 41.4s	remaining: 3m 54s
700:	test: 0.8461852	test1: 0.8163082	best: 0.8163082 (700)	total: 48.2s	remaining: 3m 47s
800:	test: 0.8478560	test1: 0.8169265	best: 0.8169494 (793)	total: 55.1s	remaining: 3m 39s
900:	test: 0.8493902	test1: 0.8173456	best: 0.8173503 (899)	total: 1m 1s	remaining: 3m 32s
1000:	test: 0.8508648	test1: 0.8176771	best: 0.8176771 (1000)	total: 1m 8s	remaining: 3m 25s
1100:	test: 0.8524996	test1: 0.8179472	best: 0.8179476 (1085)	total: 1m 15s	remaining: 3m 19s
1200:	test: 0.8541343	test1: 0.8182771	best: 0.8183470 (1185)	total: 1m 22s	remaining: 3m 12s
1300:	test: 0.8558301	test1: 0.8185348	best: 0.8185461 (1287)	total: 1m 29s	remaining: 3m 5s
1400:	test: 0.8574767	test1: 0.8188174	best: 0.8188189 (1397)	total: 1m 36s	remaining: 2m 58s
1500:	test: 0.8591559	test1: 0.8190681	best: 0.8190801 (1498)	total: 1m 43s	re

1200:	test: 0.8519379	test1: 0.8477167	best: 0.8477229 (1196)	total: 1m 20s	remaining: 3m 8s
1300:	test: 0.8535547	test1: 0.8482247	best: 0.8482282 (1293)	total: 1m 27s	remaining: 3m 1s
1400:	test: 0.8552491	test1: 0.8484968	best: 0.8484968 (1400)	total: 1m 33s	remaining: 2m 54s
1500:	test: 0.8568701	test1: 0.8487425	best: 0.8487425 (1500)	total: 1m 40s	remaining: 2m 47s
1600:	test: 0.8585041	test1: 0.8488725	best: 0.8488958 (1594)	total: 1m 47s	remaining: 2m 40s
1700:	test: 0.8601360	test1: 0.8489660	best: 0.8489982 (1693)	total: 1m 53s	remaining: 2m 33s
1800:	test: 0.8616424	test1: 0.8490747	best: 0.8490747 (1800)	total: 2m	remaining: 2m 27s
1900:	test: 0.8631884	test1: 0.8490017	best: 0.8490836 (1806)	total: 2m 7s	remaining: 2m 20s
2000:	test: 0.8646017	test1: 0.8490409	best: 0.8490836 (1806)	total: 2m 14s	remaining: 2m 13s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8490836253
bestIteration = 1806

Shrink model to first 1807 iterations.
error:  0.8148941869

In [30]:
#file for submission
id = test_given['Applicant_ID']
#id

sub = pd.DataFrame(id, columns=['Applicant_ID'])
sub['default_status'] = preds_proba
sub

sub.to_csv('DSN_Hack_ear(cat-999stra15_).csv', index=False)

In [13]:
from sklearn.metrics import roc_auc_score

def metric(Y,pred):
    return roc_auc_score(Y,pred, labels=[0,1])

In [14]:
from sklearn.model_selection import StratifiedKFold

n_splits = 10
params_3 = {'n_estimators':4000, 'learning_rate':0.01, 'objective':'CrossEntropy','eval_metric':'AUC',
          'random_seed': 1234,'early_stopping_rounds':200,'use_best_model':True}
fold=StratifiedKFold(n_splits)


In [16]:
#Best
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

list_scores = []
test_oofs = []
score = 0 

for i, (tr_idx, vr_idx) in enumerate(fold.split(X,Y)):
    X_train, Y_train = X.loc[tr_idx, new_features], Y.loc[tr_idx]
    X_val, Y_val = X.loc[vr_idx,new_features], Y.loc[vr_idx]
    
    model= CatBoostClassifier(**params_3)
    model.fit(X_train,Y_train,eval_set=[(X_val,Y_val)],verbose=100,)#erly100
    
    pred = model.predict_proba(X_val)[:,1]
    sc = metric(Y_val, pred)
    list_scores.append(sc)
    score += sc/n_splits
    
    pred_test = model.predict_proba(test_given_new[test_new_features])[:,1]
    test_oofs.append(pred_test)

0:	test: 0.7847852	best: 0.7847852 (0)	total: 112ms	remaining: 7m 29s
100:	test: 0.8166772	best: 0.8166772 (100)	total: 6.38s	remaining: 4m 6s
200:	test: 0.8207474	best: 0.8207584 (199)	total: 11.5s	remaining: 3m 36s
300:	test: 0.8231995	best: 0.8231995 (300)	total: 16.6s	remaining: 3m 23s
400:	test: 0.8250217	best: 0.8250217 (400)	total: 21.8s	remaining: 3m 15s
500:	test: 0.8261680	best: 0.8261680 (500)	total: 27.1s	remaining: 3m 9s
600:	test: 0.8272584	best: 0.8272584 (600)	total: 32.4s	remaining: 3m 3s
700:	test: 0.8281952	best: 0.8281952 (700)	total: 37.8s	remaining: 2m 57s
800:	test: 0.8288637	best: 0.8288637 (800)	total: 43.2s	remaining: 2m 52s
900:	test: 0.8293920	best: 0.8294012 (899)	total: 48.5s	remaining: 2m 46s
1000:	test: 0.8299665	best: 0.8299676 (999)	total: 53.8s	remaining: 2m 41s
1100:	test: 0.8305264	best: 0.8305264 (1100)	total: 59.5s	remaining: 2m 36s
1200:	test: 0.8310499	best: 0.8310499 (1200)	total: 1m 4s	remaining: 2m 31s
1300:	test: 0.8313689	best: 0.8313751 (1

0:	test: 0.7981384	best: 0.7981384 (0)	total: 104ms	remaining: 6m 55s
100:	test: 0.8277452	best: 0.8277452 (100)	total: 6.68s	remaining: 4m 17s
200:	test: 0.8324015	best: 0.8324015 (200)	total: 12.2s	remaining: 3m 50s
300:	test: 0.8349112	best: 0.8349112 (300)	total: 17.4s	remaining: 3m 34s
400:	test: 0.8366179	best: 0.8366196 (398)	total: 22.9s	remaining: 3m 25s
500:	test: 0.8378778	best: 0.8378778 (500)	total: 28.5s	remaining: 3m 19s
600:	test: 0.8387277	best: 0.8387277 (600)	total: 34s	remaining: 3m 12s
700:	test: 0.8395221	best: 0.8395221 (700)	total: 39.6s	remaining: 3m 6s
800:	test: 0.8401717	best: 0.8401774 (799)	total: 45.4s	remaining: 3m 1s
900:	test: 0.8407524	best: 0.8407619 (899)	total: 50.8s	remaining: 2m 54s
1000:	test: 0.8412612	best: 0.8412671 (999)	total: 56.1s	remaining: 2m 48s
1100:	test: 0.8415620	best: 0.8415655 (1092)	total: 1m 1s	remaining: 2m 41s
1200:	test: 0.8419325	best: 0.8419325 (1200)	total: 1m 6s	remaining: 2m 35s
1300:	test: 0.8421119	best: 0.8421119 (13

0:	test: 0.8168655	best: 0.8168655 (0)	total: 108ms	remaining: 7m 11s
100:	test: 0.8450812	best: 0.8450850 (99)	total: 6.49s	remaining: 4m 10s
200:	test: 0.8479479	best: 0.8479479 (200)	total: 11.6s	remaining: 3m 40s
300:	test: 0.8497419	best: 0.8497419 (300)	total: 16.7s	remaining: 3m 25s
400:	test: 0.8511132	best: 0.8511132 (400)	total: 21.9s	remaining: 3m 16s
500:	test: 0.8520110	best: 0.8520110 (500)	total: 27.2s	remaining: 3m 9s
600:	test: 0.8527518	best: 0.8527518 (600)	total: 32.4s	remaining: 3m 3s
700:	test: 0.8531398	best: 0.8531441 (698)	total: 38s	remaining: 2m 58s
800:	test: 0.8534480	best: 0.8534492 (799)	total: 43.8s	remaining: 2m 54s
900:	test: 0.8536320	best: 0.8536470 (889)	total: 49.8s	remaining: 2m 51s
1000:	test: 0.8537875	best: 0.8537902 (993)	total: 55.1s	remaining: 2m 45s
1100:	test: 0.8539368	best: 0.8539481 (1099)	total: 1m	remaining: 2m 38s
1200:	test: 0.8539738	best: 0.8540159 (1181)	total: 1m 5s	remaining: 2m 32s
1300:	test: 0.8538856	best: 0.8540159 (1181)	

In [17]:
oof_pred = pd.DataFrame(test_oofs).T
oof_pred.columns = ['split_' + str(i) for i in range(1, n_splits+1)]
oof_pred.head()

,split_1,split_2,split_3,split_4,split_5,split_6,split_7,split_8,split_9,split_10
0,0.393187,0.406448,0.399126,0.401955,0.372573,0.425641,0.393528,0.395876,0.377332,0.393702
1,0.266739,0.289575,0.278074,0.294909,0.383767,0.258905,0.304176,0.237468,0.335788,0.258510
2,0.362215,0.358191,0.385801,0.381403,0.387808,0.429348,0.403883,0.409981,0.341807,0.372387
3,0.742730,0.752221,0.765588,0.768063,0.748520,0.774655,0.766618,0.772214,0.750292,0.748555
4,0.184959,0.217812,0.203541,0.203265,0.211855,0.193568,0.189801,0.178006,0.225564,0.186233


In [21]:
sample_submission['default_status'] = np.mean(test_oofs, axis = 0)

sample_submission.to_csv('Submission_new.csv', index=False)

In [ ]:
from sklearn.metrics import roc_auc_score

def metric(Y,pred):
    return roc_auc_score(Y,pred, labels=[0,1])

from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

list_scores = []
test_oofs = []
score = 0 

for i, (tr_idx, vr_idx) in enumerate(fold.split(X,Y)):
    X_train, Y_train = X.loc[tr_idx, new_features], Y.loc[tr_idx]
    X_val, Y_val = X.loc[vr_idx,new_features], Y.loc[vr_idx]
    
    model= CatBoostClassifier(**params_3)
    model.fit(X_train,Y_train,eval_set=[(X_val,Y_val)],verbose=100,)#erly100
    
    pred = model.predict_proba(X_val)[:,1]
    sc = metric(Y_val, pred)
    list_scores.append(sc)
    score += sc/n_splits
    
    pred_test = model.predict_proba(test_given_new[test_new_features])[:,1]
    test_oofs.append(pred_test)